# Notebook 07 — Rolling Window Analysis

## Time-Varying OU Parameters (24-Month Rolling Window)

Notebook 03 estimated a single set of OU parameters for each currency over the entire 13-year sample. But the KHR spread compressed from ~24% to ~5%, meaning the "true" parameters changed dramatically over time. This notebook tracks parameter evolution by re-estimating the OU model on a **rolling 24-month window**.

**Why 24 months?** It balances signal vs. noise — enough observations (24) for reasonable MLE estimation, but short enough to capture structural shifts like the KHR compression.

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.figsize': (12, 6), 'figure.dpi': 150, 'savefig.dpi': 300,
    'font.size': 11, 'axes.titlesize': 14, 'axes.labelsize': 12,
    'legend.fontsize': 10, 'font.family': 'serif'
})
print('Libraries loaded.')

In [ ]:
# ─── Load Data ───────────────────────────────────────────────────────────────
usd = pd.read_csv('../data/processed/spreads_usd_new_amount.csv', parse_dates=['date'], index_col='date')
khr = pd.read_csv('../data/processed/spreads_khr_new_amount.csv', parse_dates=['date'], index_col='date')
S_usd = usd['spread'].values
S_khr = khr['spread'].values
dates = usd.index
dt = 1/12
window = 24

def ou_neg_log_likelihood(params, data, dt):
    kappa, theta, sigma = params
    if kappa <= 0 or sigma <= 0: return 1e10
    n = len(data) - 1
    exp_kdt = np.exp(-kappa * dt)
    m = theta + (data[:-1] - theta) * exp_kdt
    v = (sigma**2 / (2 * kappa)) * (1 - np.exp(-2 * kappa * dt))
    if v <= 0: return 1e10
    residuals = data[1:] - m
    ll = -0.5 * n * np.log(2 * np.pi) - 0.5 * n * np.log(v) - 0.5 * np.sum(residuals**2) / v
    return -ll

def estimate_ou(data, dt):
    best_nll, best_x = np.inf, None
    for k0 in [0.5, 1.0, 2.0, 5.0, 10.0, 20.0]:
        result = minimize(ou_neg_log_likelihood, [k0, np.mean(data), np.std(np.diff(data))*np.sqrt(12)],
                         args=(data, dt), method='Nelder-Mead', options={'maxiter': 50000, 'xatol': 1e-10, 'fatol': 1e-10})
        if result.fun < best_nll and result.x[0] > 0 and result.x[2] > 0:
            best_nll, best_x = result.fun, result.x
    if best_x is None: return {'kappa': np.nan, 'theta': np.nan, 'sigma': np.nan}
    return {'kappa': best_x[0], 'theta': best_x[1], 'sigma': best_x[2]}

total_windows = len(S_usd) - window
print(f'Rolling window: {window} months')
print(f'Number of rolling estimates: {total_windows}')

In [ ]:
# ─── Rolling Window Estimation ───────────────────────────────────────────────
rolling_usd = {'kappa': [], 'theta': [], 'sigma': [], 'date': []}
rolling_khr = {'kappa': [], 'theta': [], 'sigma': [], 'date': []}

for i in range(total_windows):
    if i % 20 == 0:
        print(f'  Window {i+1}/{total_windows}...')
    
    est_usd = estimate_ou(S_usd[i:i+window], dt)
    est_khr = estimate_ou(S_khr[i:i+window], dt)
    
    for key in ['kappa', 'theta', 'sigma']:
        rolling_usd[key].append(est_usd[key])
        rolling_khr[key].append(est_khr[key])
    rolling_usd['date'].append(dates[i + window - 1])
    rolling_khr['date'].append(dates[i + window - 1])

roll_usd_df = pd.DataFrame(rolling_usd).set_index('date')
roll_khr_df = pd.DataFrame(rolling_khr).set_index('date')
print(f'\nDone: {total_windows} windows estimated.')

In [ ]:
# ─── FIGURE 10: Rolling Parameter Evolution ──────────────────────────────────
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

params_config = [
    ('theta', 'θ — Long-Run Equilibrium Spread (%)', 'θ'),
    ('kappa', 'κ — Mean Reversion Speed', 'κ'),
    ('sigma', 'σ — Volatility', 'σ'),
]

for ax, (param, title, symbol) in zip(axes, params_config):
    ax.plot(roll_usd_df.index, roll_usd_df[param], color='#1565C0', linewidth=1.3, label=f'{symbol} (USD)', alpha=0.9)
    ax.plot(roll_khr_df.index, roll_khr_df[param], color='#C62828', linewidth=1.3, label=f'{symbol} (KHR)', alpha=0.9)
    ax.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'), alpha=0.1, color='grey')
    ax.set_title(title, fontweight='bold', fontsize=12)
    ax.set_ylabel(symbol)
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(True, alpha=0.3)

for date, label in [('2020-03-01','COVID'), ('2022-03-01','Fed\nHikes'), ('2024-09-01','Fed\nCuts')]:
    for ax in axes:
        ax.axvline(x=pd.Timestamp(date), color='grey', linestyle=':', alpha=0.4, linewidth=0.8)

axes[2].set_xlabel('Date')
axes[2].xaxis.set_major_locator(mdates.YearLocator())
axes[2].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)

fig.suptitle('Figure 10: Rolling 24-Month OU Parameter Evolution', fontweight='bold', fontsize=14, y=1.01)
plt.tight_layout()
plt.savefig('../figures/fig10_rolling_parameters.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig10_rolling_parameters.png')

### Interpretation — Figure 10: Rolling Parameter Evolution

Figure 10 is arguably the **most informative figure** in the paper because it reveals the *time-varying* nature of credit risk dynamics that full-sample estimates conceal:

**Panel 1 — θ (Equilibrium Spread):**
- **KHR θ** (red line) shows a dramatic **monotonic decline** from ~20–24% (early windows) to ~5% (recent windows). This is the rolling window's version of the structural compression story — each 2-year window captures a successively lower equilibrium. The decline was steepest in 2017–2019 as the KHR lending market matured.
- **USD θ** (blue line) shows a more moderate decline from ~8–10% to ~5% with some variation. The post-COVID period shows θ dipping below 5%, reflecting the compressed-spread environment.
- By 2023–2025, the two θ lines have **converged to within ~1 pp** — a visual proof of the cross-currency equilibrium convergence.

**Panel 2 — κ (Mean Reversion Speed):**
- κ values are **noisy** across windows, which is expected — mean reversion speed is the hardest OU parameter to estimate precisely with only 24 observations.
- Despite the noise, some patterns emerge: periods of high κ indicate **faster self-correction** of spread deviations, while low κ periods indicate more persistent deviations.
- Watch for **spikes** around structural break dates, where the rolling window straddles two regimes.

**Panel 3 — σ (Volatility):**
- **KHR σ** shows a dramatic decline from ~5–10 (early windows) to ~1–2 (recent windows), confirming the volatility compression identified in Notebook 06. The KHR market has become **qualitatively less volatile** over time.
- **USD σ** shows a more moderate decline with a notable dip during the COVID period (windows centered on 2020–2021 show the lowest σ values) — the NBC restructuring effect.
- Post-COVID, both σ lines have **converged** to similar low levels (~1–2), suggesting the two segments now have comparable volatility profiles.

**Critical Insight for the Paper:** The rolling window analysis resolves the tension between the ADF test results (Notebook 02) and the OU model. The full-sample OU parameters are "averages" across multiple regimes, which is why the ADF test fails — the process is mean-reverting, but around a **shifting** equilibrium. The rolling window captures these shifts explicitly.

In [ ]:
# ─── Convergence Analysis ────────────────────────────────────────────────────
theta_ratio = roll_khr_df['theta'] / roll_usd_df['theta']
sigma_ratio = roll_khr_df['sigma'] / roll_usd_df['sigma']

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 7), sharex=True)

ax1.plot(theta_ratio.index, theta_ratio, color='#4A148C', linewidth=1.3)
ax1.axhline(y=1, color='grey', linestyle='--', alpha=0.5)
ax1.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'), alpha=0.1, color='grey')
ax1.set_ylabel('θ_KHR / θ_USD')
ax1.set_title('Rolling Ratio: θ_KHR / θ_USD (Equilibrium Convergence)', fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.plot(sigma_ratio.index, sigma_ratio, color='#E65100', linewidth=1.3)
ax2.axhline(y=1, color='grey', linestyle='--', alpha=0.5)
ax2.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'), alpha=0.1, color='grey')
ax2.set_xlabel('Date')
ax2.set_ylabel('σ_KHR / σ_USD')
ax2.set_title('Rolling Ratio: σ_KHR / σ_USD (Risk Convergence)', fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.xaxis.set_major_locator(mdates.YearLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('../figures/fig10b_convergence.png', dpi=300, bbox_inches='tight')
plt.show()

print(f'\nθ ratio: start = {theta_ratio.iloc[0]:.2f} → end = {theta_ratio.iloc[-1]:.2f}')
print(f'σ ratio: start = {sigma_ratio.iloc[0]:.2f} → end = {sigma_ratio.iloc[-1]:.2f}')
if theta_ratio.iloc[-1] < theta_ratio.iloc[0]:
    print('→ USD and KHR equilibrium spreads are CONVERGING.')
else:
    print('→ USD and KHR equilibrium spreads are DIVERGING.')

### Interpretation — Convergence / Divergence Analysis

The ratio plots track whether the two currency segments are becoming **more or less similar** over time:

**θ_KHR / θ_USD Ratio:**
- Started at ~2.5–3.0× (the KHR equilibrium was 2.5–3 times the USD equilibrium in the early windows)
- **Declined monotonically** toward ~1.0 by the most recent windows
- A ratio near 1.0 means the two currencies have **similar equilibrium spreads** — the historical KHR exchange rate risk premium has effectively disappeared in the term lending market
- The grey dashed line at ratio = 1.0 represents **full convergence**; the KHR/USD θ ratio is now very close to this line

**σ_KHR / σ_USD Ratio:**
- Started at ~2–3× (KHR was 2–3 times more volatile)
- Shows more variation than the θ ratio, with periods of increasing and decreasing divergence
- The trend is generally **downward** but with noise — KHR volatility relative to USD has declined but not as smoothly as the equilibrium convergence

**De-Dollarization Implications:**
This convergence is the **quantitative evidence** for the success of NBC's de-dollarization efforts. As the KHR lending market matures:
- Banks require less risk premium for riel loans → θ converges
- KHR pricing becomes more stable → σ ratio declines
- If this trend continues, the dual-currency credit risk framework may eventually become unnecessary — a single-currency model would suffice when the currencies are essentially interchangeable in credit pricing

In [ ]:
# ─── Save Rolling Results ────────────────────────────────────────────────────
rolling_export = pd.DataFrame({
    'date': roll_usd_df.index,
    'kappa_usd': roll_usd_df['kappa'].values, 'theta_usd': roll_usd_df['theta'].values,
    'sigma_usd': roll_usd_df['sigma'].values,
    'kappa_khr': roll_khr_df['kappa'].values, 'theta_khr': roll_khr_df['theta'].values,
    'sigma_khr': roll_khr_df['sigma'].values,
})
rolling_export.to_csv('../data/processed/rolling_ou_parameters.csv', index=False)
print('Saved: rolling_ou_parameters.csv')

---
## Summary

| Finding | Evidence | Implication |
|---------|----------|-------------|
| θ converging toward 1:1 | θ_KHR/θ_USD fell from ~3× to ~1× | KHR exchange rate risk premium disappearing |
| σ also declining | Both currencies at low volatility | Banking sector matured, more stable pricing |
| κ noisy but positive | Rolling κ remains > 0 in most windows | Mean reversion confirmed throughout |
| Structural break visible | Sharp θ_KHR decline 2016–2019 | Full-sample estimates are misleading |
| COVID dip in σ | Temporary volatility suppression | NBC restructuring froze market dynamics |